# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.

# Resolução

## Questão 1


In [1]:
# X = [0.5, 0.6, 0.7, 1, 2, 3] => predizer a classe
# 
# p(c|x) = P(X|C)*P(C) / P(X) => P(X|C)*P(C)
# MAIOR P(C|X) ai a gente retorna que é a c

# probabilidade da classe = p(c) => len(classe)/numero de linhas do dataset
# probabilidade do elemento x dado a classe = p(x|c) = p(x[0]|c)*p(x[1]|c)*p(x[2]|c)*...*p(x[5]|c)  
# X = [0.5, 0.6, 0.7, 1, 2, 3]
# probabilidade do elemento x = p(x)

# probabilidade de cada elemento para cada feature para cada classe => elemento/len(classe)
# os valores da classe estão ordenados de 0 a 3

In [4]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from statistics import mean

In [6]:
def categorize_data(dataset, columns):
    # Categorizando os dados
    for feature in columns:
        dataset[feature] = dataset[feature].astype('category')
        dataset[feature] = dataset[feature].cat.codes
    return dataset

def createNB(x, y):
    classes = np.unique(y)
    K = classes.shape[0]
    D = x.shape[1]
    
    prioridadeClasses = np.array([np.sum(y == k) for k in classes])/y.shape[0]
    
    mean = np.zeros((D, K))
    var = np.zeros((D, K))
    for k in classes:
        xk = x[y == k]
        mean[:,k] = np.mean(xk, axis = 0)
        var[:, k] = np.sum((xk - mean[:, k])**2, axis = 0) / xk.shape[0]
    
    return {'classes': classes, 'prioridade': prioridadeClasses, 'K': K, 'D': D, 'mean': mean, 'var':var}

def predic(model, x):
    pred = np.zeros((x.shape[0], model['K']))
    
    for k in range(model['K']):
        pred[:,k] = - 0.5 * np.sum(np.log(2*np.pi*model['var'][:,k])) \
                    - 0.5 * np.sum((x - model['mean'][:,k])**2 / model['var'][:,k], axis=1) \
                    + model['prioridade'][k]
    return model['classes'][np.argmax(pred, axis=1)]

In [7]:
columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]
features = ["buying", "maint", "doors", "persons", "lug_boot", "safety"]
map_class = {0:'acc', 1:'good', 2:'unacc', 3:'vgood'}

dataset = pd.read_csv("carData.csv", names=["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"])
dataset = categorize_data(dataset, columns)

treino, teste = train_test_split(dataset,test_size=0.3)
treino_x = treino[features]
treino_y = treino['class']
teste_x = teste[features]
teste_y = teste['class']

In [10]:
model = createNB(np.array(treino_x), np.array(treino_y))
model
predict = predic(model, teste_x)
accuracy_score(teste_y, predict)

C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log


0.055876685934489405

# Questão 2 & 3

In [11]:
for feature in ["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"]:
    dataset[feature] = dataset[feature].astype('category')
    dataset[feature] = dataset[feature].cat.codes

kf = KFold(n_splits = 5, shuffle = True)
accuracy1 = [];
accuracy2 = [];

for _ in range(0,5):
    result = next(kf.split(dataset), None)
    train = dataset.iloc[result[0]]
    test =  dataset.iloc[result[1]]
    
    target_train = train['class']
    target_test = test['class']
    
    train = train[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    test = test[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    
    naive_bayes = GaussianNB()
    naive_bayes.fit(train, target_train)
    target_pred = naive_bayes.predict(test)
    
    model = createNB(np.array(train), np.array(target_train))
    predict = predic(model, test)
    
    accuracy1.append(accuracy_score(target_test, target_pred))
    accuracy2.append(accuracy_score(target_test, predict))

print(accuracy1)
print(accuracy2)
# Fazer média da acuracia

C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log


[0.653179190751445, 0.6445086705202312, 0.6329479768786127, 0.5838150289017341, 0.630057803468208]
[0.023121387283236993, 0.023121387283236993, 0.04046242774566474, 0.02023121387283237, 0.04046242774566474]


C:\Users\andre\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log


# Discussão sobre solução

Nitidamente o resultado provido pelo uso de funções do Scikit-Learn foi bem melhor do que o que implementamos a mão. Infelizmente, após refazer o algoritmo 2 vezes, ainda não pudemos observar o que está acontecendo para que sua acurácea esteja tão ruim.